In [63]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas_profiling import ProfileReport

In [64]:
def load_data(file_path, target_column):
    data = pd.read_csv(file_path)

In [65]:
profile = ProfileReport(data, title="Data Profile Report", explorative=True)
profile.to_file("data_profile_report.html")  # Exportez le rapport au format HTML
print(profile)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [73]:
data.isna().sum()
# Nous n'avons pas de valeure manquante.

country_number                     0
country_code                       0
country                            0
year                               0
systemic_crisis                    0
exch_usd                           0
domestic_debt_in_default           0
sovereign_external_debt_default    0
gdp_weighted_default               0
inflation_annual_cpi               0
independence                       0
currency_crises                    0
inflation_crises                   0
banking_crisis                     0
dtype: int64

In [66]:
def load_data(file_path, target_column):
    data = pd.read_csv(file_path)
    
    # Sélectionner les colonnes catégorielles (qualitatives) et les colonnes numériques
    categorical_columns = data.select_dtypes(include=['object']).columns
    numeric_columns = data.select_dtypes(exclude=['object']).columns
    
    # Effectuer l'encodage one-hot des variables catégorielles
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    return X, y

In [67]:
def train_random_forest_classifier(X, y):
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X, y)
    return rf_classifier

In [68]:
def train_logistic_regression_with_rf_predictions(X_train, y_train, X_test, rf_classifier):
    rf_train_predictions = rf_classifier.predict(X_train)
    X_train_with_rf_predictions = X_train.copy()
    X_train_with_rf_predictions['rf_predictions'] = rf_train_predictions

    lr_classifier = LogisticRegression(max_iter=1000, random_state=42)
    lr_classifier.fit(X_train_with_rf_predictions, y_train)

    rf_test_predictions = rf_classifier.predict(X_test)
    X_test_with_rf_predictions = X_test.copy()
    X_test_with_rf_predictions['rf_predictions'] = rf_test_predictions

    lr_predictions = lr_classifier.predict(X_test_with_rf_predictions)

    return lr_predictions

In [69]:
def evaluate_model(y_test, lr_predictions):
    accuracy = accuracy_score(y_test, lr_predictions)
    return accuracy

In [71]:
if __name__ == "__main__":
    file_path = '/Users/mac/Downloads/African_crises_dataset.csv'  # Assurez-vous de spécifier le chemin vers votre fichier CSV
    target_column = 'systemic_crisis'  # Remplacez par le nom de votre variable cible

    X, y = load_data(file_path, target_column)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_classifier = train_random_forest_classifier(X_train, y_train)
    lr_predictions = train_logistic_regression_with_rf_predictions(X_train, y_train, X_test, rf_classifier)

    accuracy = evaluate_model(y_test, lr_predictions)
    print(f"Accuracy of Logistic Regression with Random Forest predictions: {accuracy}")

Accuracy of Logistic Regression with Random Forest predictions: 1.0
